webscraping collections data and general bibliography


# query on data.bnf 

it's not possible to do a query directly using python, so this query:
```PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bnf-onto: <http://data.bnf.fr/ontology/bnf-onto/>
SELECT * 
WHERE {
  ?work dct:title ?title ;
        dct:publisher ?publisher;
        dct:date ?date;
        rdfs:seeAlso ?uri ;
        bnf-onto:isbn ?isbn
  FILTER (bif:contains(?title, "Klimt"))}```

has been done on the web endpoint and then the CSV was downloaded. 

In [6]:
import pandas as pd
bnf= pd.read_csv("databnf_KLI.csv")

bnf.head()


,work,title,publisher,date,uri,isbn
0,http://data.bnf.fr/ark:/12148/cb45532401k#about,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,https://catalogue.bnf.fr/ark:/12148/cb45532401k,0878468528
1,http://data.bnf.fr/ark:/12148/cb45532401k#about,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,https://catalogue.bnf.fr/ark:/12148/cb45532401k,9780878468522
2,http://data.bnf.fr/ark:/12148/cb35356100b#about,"Art in Vienna, 1898-1918 : Klimt, Kokoschka, S...","London : Phaidon , 1975",1975,https://catalogue.bnf.fr/ark:/12148/cb35356100b,0-7148-1600-0
3,http://data.bnf.fr/ark:/12148/cb37494522j#about,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 1993",1993,https://catalogue.bnf.fr/ark:/12148/cb37494522j,0-7148-1600-0
4,http://data.bnf.fr/ark:/12148/cb39124684p#about,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 2001",2001,https://catalogue.bnf.fr/ark:/12148/cb39124684p,0-7148-2967-6


In [7]:
for column_name in bnf.columns:
    if column_name == "isbn":
        for i, value in enumerate(bnf[column_name]):
            if "-" in value:
                # Replacing hyphens with empty string
                bnf.at[i, column_name] = value.replace("-", "")
bnf.head()

,work,title,publisher,date,uri,isbn
0,http://data.bnf.fr/ark:/12148/cb45532401k#about,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,https://catalogue.bnf.fr/ark:/12148/cb45532401k,0878468528
1,http://data.bnf.fr/ark:/12148/cb45532401k#about,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,https://catalogue.bnf.fr/ark:/12148/cb45532401k,9780878468522
2,http://data.bnf.fr/ark:/12148/cb35356100b#about,"Art in Vienna, 1898-1918 : Klimt, Kokoschka, S...","London : Phaidon , 1975",1975,https://catalogue.bnf.fr/ark:/12148/cb35356100b,0714816000
3,http://data.bnf.fr/ark:/12148/cb37494522j#about,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 1993",1993,https://catalogue.bnf.fr/ark:/12148/cb37494522j,0714816000
4,http://data.bnf.fr/ark:/12148/cb39124684p#about,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 2001",2001,https://catalogue.bnf.fr/ark:/12148/cb39124684p,0714829676


# query on google books api

In [8]:
import requests
import json

def fetch_books(query, max_results=40):
    base_url = "https://www.googleapis.com/books/v1/volumes"
    start_index = 0
    all_results = []

    while True:
        params = {
            "q": query,
            "startIndex": start_index,
            "maxResults": max_results
        }
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            data = response.json()
            items = data.get("items", [])
            if not items:
                break
            all_results.extend(items)
            start_index += max_results
        else:
            print("Failed to retrieve data. Status code:", response.status_code)
            break

    return all_results

# Search for books related to Cezanne
cezanne_books = fetch_books("Klimt")

# Saving JSON data to a file
with open("klimt_books.json", "w") as json_file:
    json.dump(cezanne_books, json_file, indent=4)

print("JSON data saved to klimt_books.json")

JSON data saved to klimt_books.json


In [9]:
import json
import pandas as pd
import numpy as np

# Load JSON data from file
with open("klimt_books.json", "r") as json_file:
    cezanne_books_data = json.load(json_file)

# Extract relevant fields from each book item
books_list = []
for book in cezanne_books_data:
    book_info = {
        "Title": book["volumeInfo"].get("title", "N/A"),
        "Subtitle": book["volumeInfo"].get("subtitle", "N/A"),
        "Authors": ", ".join(book["volumeInfo"].get("authors", ["N/A"])),
        "Publisher": book["volumeInfo"].get("publisher", "N/A"),
        "PublishedDate": book["volumeInfo"].get("publishedDate", "N/A"),
        "isbn": book["volumeInfo"].get("industryIdentifiers", [{}])[0].get("identifier", "N/A"),  # Retrieving ISBN
    }
    books_list.append(book_info)

# Create DataFrame
books_df = pd.DataFrame(books_list)
books_df.replace('N/A', np.nan, inplace=True)

# Display DataFrame
books_df.head()


,Title,Subtitle,Authors,Publisher,PublishedDate,isbn
0,Klimt,la secessione e l'Italia,NaN,NaN,2021,8857246574
1,Gustav Klimt,NaN,Gustav Klimt,NaN,1981,8820204649
2,Gustav Klimt. Tutti i dipinti,NaN,Tobias G. Natter,NaN,2018,383656291X
3,Klimt,NaN,"Gustav Klimt, Cecilia Bernardini",NaN,1980,UCSC:32106008329085
4,L'opera completa di Klimt,NaN,"Gustav Klimt, Johannes Dobai, Sergio Coradeschi",NaN,1978,PSU:000032538878


In [10]:
liss = []
for column_name in books_df.columns:
    if column_name == "Title":
        for value in books_df[column_name]:
            if "Klimt" in value or "Klimt" in value:
                liss.append(value)
print(len(liss))

209


In [11]:
import pandas as pd

# Assuming books_df is your DataFrame
new_dataframe = books_df[books_df["Title"].str.contains("Klimt", case=False)]

new_dataframe.reset_index(drop=True, inplace=True)

new_dataframe.head()



,Title,Subtitle,Authors,Publisher,PublishedDate,isbn
0,Klimt,la secessione e l'Italia,NaN,NaN,2021,8857246574
1,Gustav Klimt,NaN,Gustav Klimt,NaN,1981,8820204649
2,Gustav Klimt. Tutti i dipinti,NaN,Tobias G. Natter,NaN,2018,383656291X
3,Klimt,NaN,"Gustav Klimt, Cecilia Bernardini",NaN,1980,UCSC:32106008329085
4,L'opera completa di Klimt,NaN,"Gustav Klimt, Johannes Dobai, Sergio Coradeschi",NaN,1978,PSU:000032538878


In [12]:
import pandas as pd

# Assuming df1 and df2 are your two DataFrames with the same column header 'ISBN'

# Merge the two DataFrames on the 'ISBN' column
df_combined = pd.merge(bnf, new_dataframe, on='isbn', how='inner')

# Display the new DataFrame with rows where ISBN is found in both DataFrames
df_combined


,work,title,publisher,date,uri,isbn,Title,Subtitle,Authors,Publisher,PublishedDate
0,http://data.bnf.fr/ark:/12148/cb45640630m#about,Klimt lost,Wien : Czernin Verlag,2018,https://catalogue.bnf.fr/ark:/12148/cb45640630m,3707606414,Klimt Lost,Reihe Klimt Villa,"Niko Wahl, Marion Krammer",NaN,2018-04-25
1,http://data.bnf.fr/ark:/12148/cb46858502q#about,"Klimt e i maestri ""segreti"" della Ricci Oddi :...",Piacenza : Galleria d'arte moderna Ricci Oddi ...,2021,https://catalogue.bnf.fr/ark:/12148/cb46858502q,9791259580344,Klimt e i maestri «segreti» della Ricci Oddi. ...,NaN,E. Pontiggia,NaN,2021
2,http://data.bnf.fr/ark:/12148/cb46855667v#about,"Klimt : la secessione e l'Italia, [mostra al P...","Milano : Skira , 2021",2021,https://catalogue.bnf.fr/ark:/12148/cb46855667v,8857246574,Klimt,la secessione e l'Italia,NaN,NaN,2021
3,http://data.bnf.fr/ark:/12148/cb43818271g#about,"Klimt : alle origini di un mito, [mostra, Mila...","Milano : 24 ore cultura , impr. 2014",2014,https://catalogue.bnf.fr/ark:/12148/cb43818271g,8866481939,Klimt,alle origini di un mito,Alfred Weidinger,24 Ore Cultura,2014
4,http://data.bnf.fr/ark:/12148/cb442786023#about,"Klimt : up close and personal, paintings, lett...","Wien : Brandstätter , cop. 2012",2012,https://catalogue.bnf.fr/ark:/12148/cb442786023,3850336298,Klimt,"Up Close and Personal : Paintings, Letters, In...","Gustav Klimt, Tobias Günter Natter, Franz Smol...",Christian Brandstatter,2012
5,http://data.bnf.fr/ark:/12148/cb43812128m#about,"L'ossessione nordica : Böcklin, Klimt, Munch ...","Venezia : Marsilio , 2014",2014,https://catalogue.bnf.fr/ark:/12148/cb43812128m,8831717847,"L'ossessione nordica. Klimt, Böcklin, Munch e ...",NaN,"Giandomenico Romanelli, A. Tiddia",NaN,2014
6,http://data.bnf.fr/ark:/12148/cb472430726#about,"Klimt e l'arte italiana : [mostra, Museo d'art...","Cinisello Balsamo, Milano : Silvana editoriale",2023,https://catalogue.bnf.fr/ark:/12148/cb472430726,8836654703,Klimt e l'arte italiana. Ediz. illustrata,NaN,"Beatrice Avanzi, V. Sgarbi",NaN,2023
7,http://data.bnf.fr/ark:/12148/cb40941041p#about,"Schiele, Klimt, Kokoschka e gli amici viennesi...","Milano : Skira ; Trento : MART, Museo di arte ...",2006,https://catalogue.bnf.fr/ark:/12148/cb40941041p,8876249508,"Schiele, Klimt, Kokoschka e gli amici viennesi...",NaN,"Tobias Günter Natter, Tomas Sharman, Thomas Tr...",NaN,2006


In [13]:
import pandas as pd

# Assuming df1 and df2 are your two DataFrames

# Rename the 'Title' column in df2 to 'title'
new_dataframe.rename(columns={'Title': 'title'}, inplace=True)
new_dataframe.rename(columns={'PublishedDate': 'date'}, inplace=True)
new_dataframe.rename(columns={'Publisher': 'publisher'}, inplace=True)
# Concatenate the DataFrames vertically
combined_df = pd.concat([bnf, new_dataframe], ignore_index=True)

# Drop duplicates based on 'isbn' column
new_df = combined_df.drop_duplicates(subset='isbn')
new_df = combined_df.drop_duplicates(subset=['title', 'publisher', 'date'], keep='first')
# Reset index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)
new_df.drop(columns=['work', 'uri'], inplace=True)
index_column = new_df.columns.get_loc('Subtitle')

# Move the column to position 2
new_column_order = list(new_df.columns)
new_column_order.insert(1, new_column_order.pop(index_column))
new_df = new_df[new_column_order]

for i, date_value in enumerate(new_df['date']):
    # Convert integer to string before splitting
    date_str = str(date_value)
    # Split the date string by '-'
    date_components = date_str.split('-')[0]
    # Update the 'date' column with the list of components
    new_df.at[i, 'date'] = date_components
    
for i in new_df['Subtitle']:
    if pd.isna(i):
        new_df['Subtitle'].replace({pd.NA: ''}, inplace=True)

# Merge the values of 'title' and 'Subtitle' columns
new_df['title'] = new_df['title'] + '. ' + new_df['Subtitle']
new_df.drop(columns=['Subtitle'], inplace=True)

new_df.to_csv('bibliographic_data_KLI.csv', index=False)
# Display the new DataFrame with unique rows based on ISBN and consistent column name 'title'
new_df


C:\Users\crosi\AppData\Local\Temp\ipykernel_15084\405064620.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataframe.rename(columns={'Title': 'title'}, inplace=True)
C:\Users\crosi\AppData\Local\Temp\ipykernel_15084\405064620.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dataframe.rename(columns={'PublishedDate': 'date'}, inplace=True)
C:\Users\crosi\AppData\Local\Temp\ipykernel_15084\405064620.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,title,publisher,date,isbn,Authors
0,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,0878468528,NaN
1,"Art in Vienna, 1898-1918 : Klimt, Kokoschka, S...","London : Phaidon , 1975",1975,0714816000,NaN
2,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 1993",1993,0714816000,NaN
3,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 2001",2001,0714829676,NaN
4,"Art in Vienna 1898-1918 : Klimt, Kokoschka, Sc...","London : Phaidon , 2015",2015,0714868787,NaN
...,...,...,...,...,...
318,Tout l'œuvre peint de Klimt.,NaN,1983,UOM:39015042493455,"Gustav Klimt, Sergio Coradeschi"
319,Gustav Klimt. La storia illustrata dei grandi ...,24Ore Cultura,2020,9788866484851,"Stefano Zuffi, Massimiliano Aurelio"
320,"Gustav Klimt, Drawings.",NaN,1983,UOM:39015027868796,"Gustav Klimt, Serge Sabarsky"
321,Klimt.,Giunti Editore,1988,8809760832,Eva Di Stefano


In [14]:
import pandas as pd

# Assuming df is your DataFrame

# Count total rows of the DataFrame
total_rows = len(new_df)

# Count rows where 'title', 'subtitle', or 'description' contain specified keywords
keyword_rows = new_df[new_df['title'].str.contains(r'exhibition|exhib\.|mostra|catalogue|catalogo|exposition|retrospective|Ausstellung', case=False, na=False) ]

# Get the count of rows containing the specified keywords
keyword_rows_count = len(keyword_rows)

print("Total rows in DataFrame:", total_rows)
print("Rows containing specified keywords:", keyword_rows_count)


Total rows in DataFrame: 323
Rows containing specified keywords: 36


In [15]:
import pandas as pd

# Assuming df is your original DataFrame

# Create a mask for rows containing specified keywords in 'title', 'subtitle', or 'description' columns
mask = new_df['title'].str.contains(r'exhibition|exhib\.|mostra|catalogue |catalogo|exposition|retrospective|Ausstellung', case=False, na=False)  

# Create the exhibitions DataFrame containing rows where keywords are present
exhibitions_dataframe = new_df[mask]

# Remove the rows where keywords are present from the original DataFrame
dfbooks = new_df[~mask]

# Reset index of the original DataFrame
dfbooks.reset_index(drop=True, inplace=True)

# Reset index of the exhibitions DataFrame
exhibitions_dataframe.reset_index(drop=True, inplace=True)
exhibitions_dataframe.to_csv('exhibitions_data_KLI.csv', index=False)

# Display the exhibitions DataFrame containing rows where keywords are present
print("\nExhibitions DataFrame:")
exhibitions_dataframe



Exhibitions DataFrame:


,title,publisher,date,isbn,Authors
0,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,0878468528,NaN
1,"Gustav Klimt : papiers érotiques, [exposition,...",[Paris] : Gallimard : Fondation Dina Vierny-Mu...,2005,2070118053,NaN
2,"Il simbolismo : da Moreau a Gauguin a Klimt, m...","Ferrara : Ferrara arte , impr. 2007",2007,8889793066,NaN
3,Heiliger Frühling : Gustav Klimt und die Anfän...,"Wien ; München : C. Brandstätter , 1999",1999,3854478569,NaN
4,"Klimt e i maestri ""segreti"" della Ricci Oddi :...",Piacenza : Galleria d'arte moderna Ricci Oddi ...,2021,9791259580344,NaN
5,"Klimt, Kokoschka, Schiele : dall'art nouveau a...","Milano : Mazzotta , 2001",2001,8820215020,NaN
6,"Vienna 1900 : Klimt, Schiele, Moser, Kokoschka...","Paris : Réunion des musées nationaux , impr. 2005",2005,271185003X,NaN
7,"Klimt : la secessione e l'Italia, [mostra al P...","Milano : Skira , 2021",2021,8857246574,NaN
8,"De la scène au tableau : David, Füssli, Klimt,...",Paris : Skira-Flammarion ; [Marseille] : MM ; ...,2009,9782081236912,NaN
9,"Vienna 1900 : Klimt, Schiele, and their times,...","Ostfildern (Allemagne) : Hatje Cantz , cop. 2010",2010,3775726853,NaN


In [16]:
import pandas as pd
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

def extract_location(text):
    doc = nlp(text)
    locations = [entity.text for entity in doc.ents if entity.label_ == 'GPE']
    return locations


# Apply the extract_location function to the 'title' and 'Subtitle' columns and store the result in a new column 'locations'
exhibitions_dataframe['Exhibit locations'] = exhibitions_dataframe.apply(lambda row: extract_location(row['title']), axis=1)

# Convert the list of extracted locations into a comma-separated string
exhibitions_dataframe['Exhibit locations'] = exhibitions_dataframe['Exhibit locations'].apply(lambda x: ', '.join(x))

exhibitions_dataframe.to_csv('exhibitions_data_KLI.csv', index=False)
exhibitions_dataframe

C:\Users\crosi\AppData\Local\Temp\ipykernel_15084\1047692575.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exhibitions_dataframe['Exhibit locations'] = exhibitions_dataframe.apply(lambda row: extract_location(row['title']), axis=1)
C:\Users\crosi\AppData\Local\Temp\ipykernel_15084\1047692575.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exhibitions_dataframe['Exhibit locations'] = exhibitions_dataframe['Exhibit locations'].apply(lambda x: ', '.join(x))


,title,publisher,date,isbn,Authors,Exhibit locations
0,Klimt and Schiele : drawings from the Albertin...,"Boston, MA : Museum of Fine Arts, Boston : D.A...",2018,0878468528,NaN,"Vienna, Boston"
1,"Gustav Klimt : papiers érotiques, [exposition,...",[Paris] : Gallimard : Fondation Dina Vierny-Mu...,2005,2070118053,NaN,Paris
2,"Il simbolismo : da Moreau a Gauguin a Klimt, m...","Ferrara : Ferrara arte , impr. 2007",2007,8889793066,NaN,"Ferrara, Ferrara, giugno-16"
3,Heiliger Frühling : Gustav Klimt und die Anfän...,"Wien ; München : C. Brandstätter , 1999",1999,3854478569,NaN,"Farbe, Ausstellung"
4,"Klimt e i maestri ""segreti"" della Ricci Oddi :...",Piacenza : Galleria d'arte moderna Ricci Oddi ...,2021,9791259580344,NaN,"Piacenza, Galleria, Ricci Oddi"
5,"Klimt, Kokoschka, Schiele : dall'art nouveau a...","Milano : Mazzotta , 2001",2001,8820215020,NaN,"Kokoschka, Roma, Vittoriano"
6,"Vienna 1900 : Klimt, Schiele, Moser, Kokoschka...","Paris : Réunion des musées nationaux , impr. 2005",2005,271185003X,NaN,"Vienna, Kokoschka, Grand Palais, Paris"
7,"Klimt : la secessione e l'Italia, [mostra al P...","Milano : Skira , 2021",2021,8857246574,NaN,Rome
8,"De la scène au tableau : David, Füssli, Klimt,...",Paris : Skira-Flammarion ; [Marseille] : MM ; ...,2009,9782081236912,NaN,"Lautrec, Degas, Marseille, Toronto, juin-26"
9,"Vienna 1900 : Klimt, Schiele, and their times,...","Ostfildern (Allemagne) : Hatje Cantz , cop. 2010",2010,3775726853,NaN,"Vienna, Basel"
